<a href="https://colab.research.google.com/github/sampathk-hps/langchain-fundamentals-colab/blob/main/LangChain_1_Build_a_simple_LLM_application_with_chat_models_and_prompt_templates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installation

In [ ]:
pip install langchain

In [ ]:
pip install python-dotenv

## LangSmith

Many of the applications you build with LangChain will contain multiple steps with multiple invocations of LLM calls. As these applications get more and more complex, it becomes crucial to be able to inspect what exactly is going on inside your chain or agent. The best way to do this is with LangSmith.

1. Add keys direclty by creating the file

In [ ]:
with open('.env', 'w') as f:
    f.write('LANGSMITH_TRACING="true"\n')
    f.write('LANGSMITH_API_KEY="enter_your_key"\n')
    f.write('LANGSMITH_PROJECT="default"\n')

2. Add keys by using python_dotenv

In [ ]:
import os
import getpass

try:
  from dotenv import load_dotenv
  load_dotenv()
except ImportError:
  pass

os.environ["LANGSMITH_TRACING"] = "true"
if "LANGSMITH_API_KEY" not in os.environ:
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass(
        prompt="Enter your LangSmith API key (optional): "
    )
if "LANGSMITH_PROJECT" not in os.environ:
    os.environ["LANGSMITH_PROJECT"] = getpass.getpass(
        prompt='Enter your LangSmith Project Name (default = "default"): '
    )
    if not os.environ.get("LANGSMITH_PROJECT"):
        os.environ["LANGSMITH_PROJECT"] = "default"

In [ ]:
print(os.environ.get('LANGSMITH_TRACING'))
print(os.environ.get('LANGSMITH_API_KEY'))
print(os.environ.get('LANGSMITH_PROJECT'))

## Using LLM

In [ ]:
pip install -qU "langchain-perplexity"

In [ ]:
if not os.environ.get("PPLX_API_KEY"):
  os.environ["PPLX_API_KEY"] = getpass.getpass("Enter API key for Perplexity: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("sonar", model_provider="perplexity")

Let's first use the model directly. ChatModels are instances of LangChain Runnables, which means they expose a standard interface for interacting with them. To simply call the model, we can pass in a list of messages to the .invoke method.

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [SystemMessage(content="You are a helpful assistant."), HumanMessage(content="What is the capital of France?")]

model.invoke(messages)

AIMessage(content='The **capital of France is Paris**. It is the largest city in France and a major cultural, commercial, and political center[1][2][3].\n\nParis has a rich history as the capital, officially becoming so in 508 under King Clovis and retaining this status except for a brief period during World War II when Vichy was the capital under German occupation[1][3]. Known as the "City of Light" and "City of Love," Paris is famous for its historic landmarks such as the Eiffel Tower, Louvre Museum, Notre Dame Cathedral, and its influence in fashion, gastronomy, and the arts[1][5][6].\n\nParis is located in the north-central part of France along the Seine River and remains the political and administrative heart of the country[2][6].', additional_kwargs={'citations': ['https://home.adelphi.edu/~ca19535/page%204.html', 'https://www.britannica.com/place/France', 'https://en.wikipedia.org/wiki/List_of_capitals_of_France', 'https://www.youtube.com/watch?v=6qY9QHrJOng', 'https://www.iroam

In [ ]:
messages = [
    SystemMessage(content="Translate the following from English into Kannada"),
    HumanMessage(content="hi!"),
]

response = model.invoke(messages)
print(response.content)

"Hi!" ಅನ್ನು ಕನ್ನಡದಲ್ಲಿ "ಹಾಯ್!" ಎಂದು ಅನುವಾದಿಸುತ್ತಾರೆ.


Because chat models are Runnables, they expose a standard interface that includes async and streaming modes of invocation. This allows us to stream individual tokens from a chat model:

In [ ]:
messages = [
    SystemMessage(content="Translate the following from English into Kannada"),
    HumanMessage(content="What is your name?"),
]
for token in model.stream(messages):
    print(token.content, end="|")

ನ|ಿಮ|್ಮ| ಹೆಸರು| ಏ|ನು|?||

Prompt templates are a concept in LangChain designed to assist with the transformation - take raw user input and transform it into list of user messages ready to pass to LLM. They take in raw user input and return data (a prompt) that is ready to pass into a language model.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "You are a helpful assistant that translates from English to {output_language}."

prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("user", "{text}"),
])

# The input to this prompt template is a dictionary.
prompt = prompt_template.invoke({"output_language": "Kannada", "text": "What is the time?"})

prompt

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant that translates from English to Kannada.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is the time?', additional_kwargs={}, response_metadata={})])

In [ ]:
prompt.to_messages()

[SystemMessage(content='You are a helpful assistant that translates from English to Kannada.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is the time?', additional_kwargs={}, response_metadata={})]

In [ ]:
response = model.invoke(prompt)
print(response.content)

ಸಮಯ ಏನು? ಪ್ರಸ್ತುತ ಸಮಯವು ಕ್ಯೂಪರ್ಟಿನೊ, CAಯಲ್ಲಿ ಸಂಜೆ 9:29 PM ಆಗಿದೆ. ಇದು ಪ್ಯಾಸಿಫಿಕ್ ಡೇಲೈಟ್ ಟೈಮ್ (PDT)ನಲ್ಲಿದೆ, ಇದು UTC-7 ಗೆ ಸಮಾನವಾಗಿದೆ[2]. 

ಇದನ್ನು ಕನ್ನಡದಲ್ಲಿ ವಿವರಿಸಿದರೆ, ಸಮಯವು ಕ್ಯೂಪರ್ಟಿನೊ, CAಯಲ್ಲಿ ಸಂಜೆ 9:29 PM ಆಗಿದೆ. ಇದು ಪ್ಯಾಸಿಫಿಕ್ ಡೇಲೈಟ್ ಸಮಯ (PDT) ವಲಯದಲ್ಲಿದೆ, ಇದು UTC-7 ಗೆ ಸಮನಾಗಿದೆ. 

ನಿಮ್ಮ ಸ್ಥಳದ ಸಮಯವನ್ನು ತಿಳಿಯಲು, ನಿಮ್ಮ ನಗರದ ಸಮಯ ವಲಯವನ್ನು ತಿಳಿದುಕೊಳ್ಳಿ ಮತ್ತು ಅದನ್ನು ಪ್ಯಾಸಿಫಿಕ್ ಡೇಲೈಟ್ ಟೈಮ್ (PDT) ಜೊತೆ ಹೋಲಿಸಿ.


## Using Local LLMs

In [ ]:
!pip install ollama

In [ ]:
# Replace with the address of your remote Ollama instance
ollama_url = "http://localhost:11434"

try:
    import ollama
    client = ollama.Client(host=ollama_url)
    print(client.list())
except Exception as e:
    print(f"Could not connect to Ollama server at {ollama_url}. Please ensure a remote Ollama instance is running and accessible.")
    print(f"Error: {e}")

Could not connect to Ollama server at http://localhost:11434. Please ensure a remote Ollama instance is running and accessible.
Error: Failed to connect to Ollama. Please check that Ollama is downloaded, running and accessible. https://ollama.com/download
